In [6]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from cnn_utils import *
from PIL import Image
import glob
import random
from tensorflow.keras import layers

In [7]:
#directions = ['up', 'down', 'left', 'right']
my_list = []
path = "dataset2/*.JPG"
temp = glob.glob(path)
for image in temp:
    with open(image, 'rb') as file:
        img = Image.open(file)
        img = img.resize((224, 224))
        np_img = np.array(img)
        my_list.append(np_img)
my_list = np.array(my_list)
print(my_list.shape)

(83, 224, 224, 3)


In [8]:
# 0 up, 1 down, 2 left, 3 right, 4 zoom in, 5 zoom out, 6 two, 7 three, 8 four, 9 five
labels = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, #24
          1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1, #19
          9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9, #20
          8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8 #20
         ]
print(len(labels))
labels = np.array(labels) 

83


In [9]:
p = np.random.permutation(len(my_list))
X_orig = my_list[p]
Y_orig = labels[p]

In [10]:
X_test_orig = X_orig[0:10]
Y_test_orig = Y_orig[0:10]
X_train_orig = X_orig[11:len(labels)]
Y_train_orig = Y_orig[11:len(labels)]

In [11]:
X_train = X_train_orig/255
X_test = X_test_orig/255
Y_train = convert_to_one_hot(Y_train_orig, 10).T
Y_test = convert_to_one_hot(Y_test_orig, 10).T

In [12]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(tf.float32, [None, n_H0, n_W0, n_C0])
    Y = tf.placeholder(tf.float32, [None, n_y])
    return X, Y

def initialize_parameters():
    tf.set_random_seed(1)
    W1 = tf.get_variable("W1", [4, 4, 3, 8], initializer = tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable("W2", [2, 2, 8, 16], initializer = tf.contrib.layers.xavier_initializer(seed=0))
    parameters = {"W1": W1,"W2": W2}
    return parameters

def forward_prop(X, parameters, keep_prob = 0.5):
    W1 = parameters['W1']
    W2 = parameters['W2']
    rate=1-keep_prob
    Z1 = tf.nn.conv2d(X,W1, strides = [1,1,1,1], padding = 'SAME')
    A1 = tf.nn.relu(Z1)
    A1 = tf.nn.dropout(A1,rate=rate)
    P1 = tf.nn.max_pool(A1, ksize = [1,8,8,1], strides = [1,8,8,1], padding = 'SAME')
    Z2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
    A2 = tf.nn.relu(Z2)
    A2 = tf.nn.dropout(A2,rate=rate)
    P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')
    P2 = tf.layers.Flatten()(P2)
    Z3 = tf.keras.layers.Dense(10, activation=None)(P2)
    return Z3

def compute_cost(Z3, Y, beta=0.01):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3,labels=Y))
    regularizer = tf.nn.l2_loss(Z3)
    cost = tf.reduce_mean(cost + beta * regularizer)
    #cost = -tf.reduce_sum(Y*tf.log(tf.clip_by_value(Z3,1e-10,1.0)))
    #cost = -tf.reduce_sum(Y*tf.log(tf.clip_by_value(Z3,1e-10,1.0)))
    #labels_sum = tf.reduce_sum(Y, axis=-1)
    #print(labels)
    #softmax = tf.nn.softmax(Z3)
    #cost = tf.reduce_mean(-tf.reduce_sum(softmax * tf.log(Y), axis=-1))
    return cost

In [13]:
def model(X_train, Y_train, learning_rate = 0.009, num_epochs = 100, minibatch_size = 64, print_cost = True):
    ops.reset_default_graph()                         
    tf.set_random_seed(1)
    seed = 3                                          
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = [] 
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    parameters = initialize_parameters()
    Z3 = forward_prop(X, parameters)
    cost = compute_cost(Z3, Y)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)

        print("Training")
        
        for epoch in range(num_epochs):
            minibatch_cost = 0
            num_minibatches = int(m / minibatch_size)
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)
            for minibatch in minibatches:
                (minibatch_X, minibatch_Y) = minibatch
                _, temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y})
                minibatch_cost += temp_cost / num_minibatches

            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
  
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        #print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})

        #train_accuracy = accuracy(predict_op.eval(), correct_prediction)
        #return train_accuracy, parameters


        saver = tf.train.Saver()
        saver.save(sess, 'hand_motions.ckpt')
        sess.close()
        
    return train_accuracy, parameters


In [14]:
def test(X_test, Y_test, parameters):                                     
    (m, n_H0, n_W0, n_C0) = X_test.shape             
    n_y = Y_test.shape[1]                            
    costs = [] 
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    Z3 = forward_prop(X, parameters)
    cost = compute_cost(Z3, Y)
    init = tf.global_variables_initializer()


    print("Validation or Testing")
    with tf.Session() as sess:
        sess.run(init)
        a = sess.run(cost, {X: X_test, Y:Y_test})

        #plt.plot(np.squeeze(a))
       # plt.show()

        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))

        #test_accuracy = accuracy(predict_op, correct_prediction)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        #print(accuracy)
        test_accuracy = accuracy.eval({X: X_train, Y: Y_train})   
        saver = tf.train.Saver()
        saver.save(sess, 'hand_motions.ckpt')
        sess.close()

In [ ]:
train, parameters = model(X_train, Y_train)
print("training accuracy: ", train)
dev = test(X_dev, Y_dev, parameters)
print("validation accuracy: ", str(dev))
test = test(X_test, Y_test, parameters)
print("test accuracy: ", str(test))
print('done')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.
Training
Cost after epoch 0: 12.958005
Cost after epoch 5: 4.538974
Cost after epoch 10: 4.386367
Cost after epoch 15: 4.102324
Cost after epoch 20: 4.070923
Cost after epoch 25: 3.942859
Cost after epoch 30: 4.130031
Cost after epoch 35: 3.830316
Cost after epoch 40: 4.042243
Cost after epoch 45: 3.753193
Cost after epoch 50: 3.715553
Cost after epoch 55: 3.971550
Cost after epoch 60: 3.735210
Cost after epoch 65: 3.633503
Cost after epoch 70: 3.700934
Cost after epoch 75: 3.776154
Cost after epoch 80: 3.580024
Cost after epoch 85: 3.676985
Cost after epoch 90: 3.682090
Cost after epoch 95: 3.472745
